In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
import os

In [ ]:
# load train data
train_df = pd.read_csv("train.csv")
train_df.head()

#load sample
def load_sample(sample_id):
    path = os.path.join("train", sample_id + ".npy")
    data = np.load(path)
    return data  # shape = (3, 4096)

sample_id = train_df.iloc[0]['id']
data = load_sample(sample_id)
data.shape


In [ ]:
#visualize time series evolution

def plot_timeseries(data, sample_id, target):
    plt.figure(figsize=(14, 8))
    
    detectors = ["Hanford", "Livingston", "Virgo"]
    for i in range(3):
        plt.subplot(3, 1, i+1)
        plt.plot(data[i], linewidth=0.8)
        plt.title(f"{detectors[i]} – sample {sample_id} – target={target}")
        plt.xlabel("time (samples)")
        plt.ylabel("strain")
    
    plt.tight_layout()
    plt.show()

target = train_df.iloc[0]['target']
plot_timeseries(data, sample_id, target)


In [ ]:
# spectrograms

def plot_spectrogram(data, sample_id, target, detector_index=0):
    f, t, Sxx = spectrogram(
        data[detector_index], 
        fs=2048, 
        nperseg=128,
        noverlap=64
    )
    
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.title(f"Spectrogram – Detector {['H','L','V'][detector_index]} – {sample_id}, target={target}")
    plt.ylabel("Frequência [Hz]")
    plt.xlabel("Tempo [s]")
    plt.colorbar(label="Intensidade")
    plt.show()

for det in range(3):
    plot_spectrogram(data, sample_id, target, detector_index=det)


In [ ]:
# explore random samples

import random

def random_sample_vis():
    row = train_df.sample(1).iloc[0]
    sample_id = row.id
    target = row.target
    data = load_sample(sample_id)
    
    print(f"Sample: {sample_id}, Target: {target}")
    plot_timeseries(data, sample_id, target)
    plot_spectrogram(data, sample_id, target)

random_sample_vis()
